In [1]:
# lib import
import numpy as np
import pandas as pd
TRAINING = True

In [2]:
def generate_features(df):
    features = ['seconds_in_bucket', 'imbalance_buy_sell_flag',
               'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2', 'prev_target'
               ]

    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')

    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap','prev_target']

    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df[f'{a}_{b}_imb'] = df.eval(f'({a}-{b})/({a}+{b})')
                features.append(f'{a}_{b}_imb')

    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1)
                    min_ = df[[a,b,c]].min(axis=1)
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_

                    df[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_)
                    features.append(f'{a}_{b}_{c}_imb2')

    return df[features]

In [3]:
# fix data
def add_targets(df):
    result = df[df['seconds_in_bucket']==540].copy(deep=True)
    result['date_id'] = result['date_id'] + 1
    result = result[['stock_id', 'date_id', 'target']]
    result = result.rename(columns={'target': 'prev_target'})

    df = df.merge(result, on=['stock_id', 'date_id'], how='left')
    return df

In [4]:
def add_targets_test(tdf, rdf):
    result = rdf.groupby(['stock_id', 'date_id']).agg(
    prev_target=('revealed_target', 'last')
    ).reset_index()

    tdf = tdf.merge(result, on=['stock_id', 'date_id'], how='left')
    
    tdf['prev_target'] = tdf['prev_target'].astype('float64')
    return tdf

In [5]:
if TRAINING:
    df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
    #original size = 5237980
    
    df = df.sample(n=500000, random_state=69)
    
    df = add_targets(df)
    
    X = generate_features(df).values
    Y = df['target'].values
        
    X = X[np.isfinite(Y)]
    Y = Y[np.isfinite(Y)]
    
    index = np.arange(len(X))

In [6]:
# import lightgbm as lgb 
# import joblib
# import os


# models = []

# # test to train ratio
# N_fold = 5

# os.system('mkdir models')

# #model_path ='/kaggle/input/testing_data_set/models'
# model_path ='/kaggle/input/optimize'

# def train(model_dict, modelname='lgbm'):
#     if TRAINING:
#         model = model_dict[modelname]
#         model.fit(X[index%N_fold!=i], Y[index%N_fold!=i],
#           eval_set=(X[index%N_fold==i], Y[index%N_fold==i]),
#           early_stopping_rounds=100,
#           verbose = 10
#          )
#         models.append(model)
#         joblib.dump(model, f'./models/{modelname}_{i}.model')
#     else:
#         models.append(joblib.load(f'{model_path}/{modelname}_{i}.model'))


        
# model_dict = {
#     'lgbm': lgb.LGBMRegressor(objective='regression_l1',
#                            colsample_bytree=0.7019911744574896,
#                            learning_rate=0.022635758411078528,
#                            max_bin=511,
#                            min_child_samples=2,
#                            n_estimators=4797,
#                            num_leaves=122,
#                            reg_alpha=0.004252223402511765,
#                            reg_lambda=0.11288241427227624
#                           )
# }
# for i in range(N_fold):
#     train(model_dict, 'lgbm')

In [7]:
# import optiver2023
# env = optiver2023.make_env()
# iter_test = env.iter_test()

In [8]:
# for (test, revealed_targets, sample_prediction) in iter_test:
#     st = test.sort_values(['stock_id', 'date_id', 'seconds_in_bucket']).copy(deep=True)
#     st = add_targets_test(st, revealed_targets)
    
#     feat = generate_features(st)
    
#     pred = (np.mean([model.predict(feat.values) for model in models], 0))
#     sample_prediction['target'] = pred
#     env.predict(sample_prediction)

In [9]:
import optuna  # pip install optuna
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

In [10]:
from optuna.integration import LightGBMPruningCallback
import lightgbm as lgbm


def objective(trial, X, y):
    param_grid = {
        'metric': 'mae', 
        'random_state': 48,
#         'device': 'gpu',
#         'gpu_platform_id': 0,
#         'gpu_device_id': 0,
#         "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt"]),
        "n_estimators": trial.suggest_categorical("n_estimators", [1200]), #71 * 10 = 710
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.4),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000),
        "max_depth": trial.suggest_int("max_depth", -1, 30),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 1000),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 500),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 500),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.1, 1.0
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.1, 1.0
        ),
    }

    cv = KFold(n_splits=5, shuffle=True, random_state=69)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgbm.LGBMRegressor(objective="regression_l1", **param_grid)
        if idx == 0:
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_test, y_test)],
                eval_metric="l1",
                early_stopping_rounds=100,
                callbacks=[
                    LightGBMPruningCallback(trial, "l1")
                ],  # Add a pruning callback
            )
        else:
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_test, y_test)],
                eval_metric="l1",
                early_stopping_rounds=100,
            )
        preds = model.predict(X_test)
        cv_scores[idx] = mean_absolute_error(y_test, preds)

    return np.mean(cv_scores)

In [11]:
%%time
study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
func = lambda trial: objective(trial, X, Y)
study.optimize(func, n_trials=100)

[I 2023-11-14 19:03:30,771] A new study created in memory with name: LGBM Regressor
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=222, reg_alpha=0.0 will be ignored. Current value: lambda_l1=222
[LightGBM] [Warning] bagging_fraction is set=0.663071987084754, subsample=1.0 will be ignored. Current value: bagging_fraction=0.663071987084754
[LightGBM] [Warning] feature_fraction is set=0.35850326205504035, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.35850326205504035
[LightGBM] [Warning] min_data_in_leaf is set=771, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=771
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=6.943720630674961, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.943720630674961
[LightGBM] [Warning] lambda_l2 is set=333, reg_lambda=0.0 will be ignored. Current value: lambda_l2=333


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=222, reg_alpha=0.0 will be ignored. Current value: lambda_l1=222
[LightGBM] [Warning] bagging_fraction is set=0.663071987084754, subsample=1.0 will be ignored. Current value: bagging_fraction=0.663071987084754
[LightGBM] [Warning] feature_fraction is set=0.35850326205504035, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.35850326205504035
[LightGBM] [Warning] min_data_in_leaf is set=771, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=771
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=6.943720630674961, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.943720630674961
[LightGBM] [Warning] lambda_l2 is set=333, reg_lambda=0.0 will be ignored. Current value: lambda_l2=333
[1]	valid_0's l1: 6.37005
[2]	valid_0's l1: 6.35362
[3]	valid_0's l1: 6.33815
[4]	valid_0's l1: 6.32757
[5]	

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


...

[LightGBM] [Warning] lambda_l1 is set=25, reg_alpha=0.0 will be ignored. Current value: lambda_l1=25
[LightGBM] [Warning] bagging_fraction is set=0.9971932960515472, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9971932960515472
[LightGBM] [Warning] feature_fraction is set=0.952692024943945, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.952692024943945
[LightGBM] [Warning] min_data_in_leaf is set=690, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=690
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=0.9032055973716262, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.9032055973716262
[LightGBM] [Warning] lambda_l2 is set=239, reg_lambda=0.0 will be ignored. Current value: lambda_l2=239
[1]	valid_0's l1: 6.31783
[2]	valid_0's l1: 6.29002
[3]	valid_0's l1: 6.27622
[4]	valid_0's l1: 6.26974
[5]	va

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=25, reg_alpha=0.0 will be ignored. Current value: lambda_l1=25
[LightGBM] [Warning] bagging_fraction is set=0.9971932960515472, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9971932960515472
[LightGBM] [Warning] feature_fraction is set=0.952692024943945, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.952692024943945
[LightGBM] [Warning] min_data_in_leaf is set=690, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=690
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=0.9032055973716262, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.9032055973716262
[LightGBM] [Warning] lambda_l2 is set=239, reg_lambda=0.0 will be ignored. Current value: lambda_l2=239
[1]	valid_0's l1: 6.3626
[2]	valid_0's l1: 6.33554
[3]	valid_0's l1: 6.32283
[4]	valid_0's l1: 6.31699
[5]	val

[I 2023-11-14 20:01:44,994] Trial 99 finished with value: 6.279448471675847 and parameters: {'boosting_type': 'gbdt', 'n_estimators': 1200, 'learning_rate': 0.3728088591728096, 'num_leaves': 2992, 'max_depth': 6, 'min_data_in_leaf': 690, 'lambda_l1': 25, 'lambda_l2': 239, 'min_gain_to_split': 0.9032055973716262, 'bagging_fraction': 0.9971932960515472, 'bagging_freq': 1, 'feature_fraction': 0.952692024943945}. Best is trial 96 with value: 6.279276042018457.


CPU times: user 3h 4min 24s, sys: 4min 5s, total: 3h 8min 30s
Wall time: 58min 14s


In [12]:
print(f"\tBest value (mae): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (mae): 6.27928
	Best params:
		boosting_type: gbdt
		n_estimators: 1200
		learning_rate: 0.3809653654265678
		num_leaves: 2969
		max_depth: 6
		min_data_in_leaf: 554
		lambda_l1: 53
		lambda_l2: 219
		min_gain_to_split: 0.3583877751432669
		bagging_fraction: 0.9322675915194278
		bagging_freq: 1
		feature_fraction: 0.9656667638946049


In [13]:
optuna.visualization.plot_optimization_history(study)

In [14]:
optuna.visualization.plot_param_importances(study)